In [1]:
from functions.get_kaggle_data import *
from functions.import_to_df import *
from functions.process_data import *

In [27]:
import pandas as pd 
import pandas_profiling as pdp
import numpy as np
import pprint as pp
import seaborn as sns
from matplotlib import pyplot as plt
# from lazypredict.Supervised import LazyRegressor 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

pd.set_option('display.max_columns', None)

In [7]:
files = get_kaggle_data(
    competition="commonlitreadabilityprize"
    )

files downloaded for commonlitreadabilityprize competition
   test.csv
   train.csv
   sample_submission.csv
located in /competition_data/


In [8]:
input_data = import_to_df(files)

In [9]:
input_data.keys()

dict_keys(['test', 'train', 'sample_submission'])

In [10]:
train_df = input_data['train']

In [11]:
trim_train_df = train_df[
    ~(train_df['standard_error'] == 0) 
    & ~(train_df['target'] == 0)]

    

In [36]:
non_train_cols = ['id','url_legal', 'license', 'excerpt', 'target', 'standard_error']
target_cols = ['target']

In [22]:
processed_train_df = process_data(trim_train_df)

In [23]:
processed_test_df = process_data(input_data['test'])

In [37]:
X_train = processed_train_df[processed_train_df.columns.difference(non_train_cols)]
y_train = processed_train_df[target_cols]
X_test = processed_test_df[processed_test_df.columns.difference(non_train_cols)]

In [25]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 40]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 5, 10, 50]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

In [34]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
/Users/oliverharvey/anaconda3/envs/titanic/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


ValueError: could not convert string to float: 'CC BY-SA 3.0 and GFDL'